<a href="https://colab.research.google.com/github/emberfox205/hme-recognition/blob/main/Assignment_HME_recognition_ColabVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
import os
from google.colab import drive
from google.colab import userdata

google_drive_dataset_path = '/content/drive/MyDrive/crohme2019/'

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Google Drive mounted.")

# Check if the dataset already exists in Google Drive
if os.path.exists(google_drive_dataset_path):
    print(f"Dataset found in Google Drive at: {google_drive_dataset_path}")
    ntcuong2103_crohme2019_path = google_drive_dataset_path
else:
    print("Dataset not found in Google Drive. Downloading from Kaggle...")
    # Download the dataset from Kaggle
    ntcuong2103_crohme2019_path = kagglehub.dataset_download('ntcuong2103/crohme2019')

!ln -s {ntcuong2103_crohme2019_path} dataset
print('Data source import complete.')


Mounting Google Drive...
Mounted at /content/drive
Google Drive mounted.
Dataset found in Google Drive at: /content/drive/MyDrive/crohme2019/
Data source import complete.


In [ ]:
# NOTE: path now may be from Drive
# If it is failed, run the above cell again to fix
assert ntcuong2103_crohme2019_path == "/kaggle/input/crohme2019"


## **CROHME Temporal Sequence Classification (CROHME-CTC) Baseline**

In this project, we will build and train a recurrent neural network (RNN) for translating handwritten mathematical expressions into sequence of symbols and relations based on Symbol Relation Tree and CROHME dataset.

The model we'll be using is a simple RNN with a single layer of Bidirectional LSTM cells. We will use Connectionist Temporal Classification (CTC) loss to train the model.

The model is trained on the CROHME dataset, which contains handwritten mathematical expressions in the form of Symbol Relation Trees. The dataset is preprocessed and converted into a sequence of symbols and relations, which is used as the input to the model.

Your job is to feature engineer the input data and train the model to achieve the best performance possible. Additionally, consider implementing data augmentation techniques to enhance the training dataset.

Some features from the project:

*   Public competition dataset: CROHME https://paperswithcode.com/dataset/crohme-2019
*   Advance data preparation: time series feature extraction, collate function
*   High level API of pytorch lightning
*   Experiment tracking with Weight and Biases



## Dependencies

In [2]:
!pip install pytorch-lightning torchmetrics wandb torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.5/962.5 kB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitl

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchmetrics.text import EditDistance


import numpy as np
import pandas as pd
import os

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
import wandb

import xml.etree.ElementTree as ET
from torch.nn.utils.rnn import pad_sequence

import matplotlib.pyplot as plt
from helper import *
from collections.abc import Sequence
from typing import Literal, Optional, Union

import json

In [ ]:
# NOTE: Should not run due to changes in import procedures
# mount crohme2019 -> dataset
!ln -s /kaggle/input/crohme2019 dataset

> The above command created a symbolic link
> dataset -> /kaggle/input/crohme2019
>
> We can now access the dataset from current directory instead of /kaggle/input

In [ ]:
# This code for displaying current directory tree, you don't need to run.

!sudo apt-get install tree
!tree -L 3 dataset

### Overview of the dataset

This is the folder structure of the dataset.

```
dataset
├── crohme2019
│   └── crohme2019
│       ├── test
│       ├── train
│       └── valid
├── crohme2019_test.txt
├── crohme2019_train.txt
└── crohme2019_valid.txt
```

There are two parts:
* Inkml files: XML based files containing handwriting data (sequences of handwritten strokes) captured from the devices.
```
dataset
├── crohme2019
│   └── crohme2019
│       ├── test
│       ├── train
│       └── valid
```
* Annotation files: Containing the labels for each input handwriting samples.
```
dataset
├── crohme2019_test.txt
├── crohme2019_train.txt
└── crohme2019_valid.txt
```


> An example of inkml file, the points (x, y coords) are recorded in each trace (handwritten stroke).

```
<ink xmlns="http://www.w3.org/2003/InkML">
<traceFormat>
<channel name="X" type="decimal"/>
<channel name="Y" type="decimal"/>
</traceFormat>
...

<trace id="0">
241 123, 240 123, 239 123, 238 123, 237 123, 236 123, 236 122, 237 122, 238 122, 240 122, 242 122, 244 122, 247 122, 251 121, 254 121, 257 121, 260 120, 262 120, 265 120, 266 120, 267 120, 268 120, 269 120, 268 120
</trace>
<trace id="1">
301 123, 300 124, 299 124, 298 124, 297 124, 296 124, 295 124, 294 124, 295 124, 296 124, 297 124, 298 124, 299 123, 301 123, 302 123, 303 123, 304 122, 305 122, 306 122, 307 122, 308 122, 310 122, 311 122, 314 122, 316 121, 317 121, 319 121, 321 121, 323 121, 328 120, 330 120, 332 120, 333 120, 335 120, 336 120, 344 119, 346 119, 348 119, 350 119, 352 119, 357 119, 358 119, 360 119, 362 119, 364 119, 365 119, 370 119, 372 119, 373 119, 374 119, 375 119, 377 119, 377 118, 376 118, 374 118, 372 118
</trace>
<trace id="2">
318 89, 319 89, 320 89, 321 88, 323 87, 324 86, 325 84, 326 83, 327 82, 328 80, 329 79, 330 77, 331 75, 332 74, 333 73, 333 72, 333 73, 333 75, 333 76, 333 78, 333 80, 332 81, 332 83, 332 85, 332 86, 332 88, 332 90, 332 92, 332 93, 333 95, 333 97, 333 98, 333 100, 333 101, 333 102, 333 103, 333 104, 333 103, 333 101, 332 99
</trace>
<trace id="3">
302 148, 301 148, 302 147, 302 146, 303 145, 304 144, 305 143, 306 142, 307 140, 308 139, 308 138, 308 137, 308 136, 308 137, 308 138, 309 139, 309 140, 309 141, 309 142, 309 144, 309 146, 309 147, 309 149, 309 151, 309 153, 309 155, 309 157, 310 159, 310 162, 310 164, 310 169, 310 171, 310 172, 310 174, 311 175, 311 179, 311 180, 311 181, 311 182, 311 181
</trace>
...

</ink>

```

> An example of annotation file: left part is inkml file path, right part is the label (target) for that inkml file.

```
...
crohme2019/test/UN19_1041_em_597.inkml	- Right \sqrt Inside 2
crohme2019/test/UN19_1019_em_256.inkml	a Right n Right y
crohme2019/test/UN19_1033_em_474.inkml	V Sub n Right - Right 1 NoRel = Right \int Right d Sup n Right - Right 1 NoRel x Right \sqrt Inside h

...
```

> The label is a sequence of **symbols** and **spatial relations** between symbols (based on writing order).



## Task 1: Build Vocabulary

Builds and manages a vocabulary for converting characters (tokens) to indices (encoding) and vice-versa (decoding). This is essential for processing text data in machine learning tasks, particularly for sequence-to-sequence models like those used in handwriting recognition or mathematical expression translation.

In this project, the vocabulary is for encoding the target sequence (for example: "- Right \\sqrt Inside 2") into a sequence of indices ([5, 37, 74, 30, 10]). It is constructed from multiple annotation files (train, test, and validation sets) and includes a special blank character (this is for CTC loss). It supports encoding (tokens to indices) and decoding (indices to tokens) operations.


> _You will need to build the `class Vocab` using the two functions provided. Also the vocabulary should be exported as a JSON file so that we can reuse it later in the dataset._

***Input***: the path to the dataset files _(train, valid, and test)_.


```python
paths = [
    "dataset/crohme2019_train.txt",
    "dataset/crohme2019_test.txt",
    "dataset/crohme2019_valid.txt",
]
```

***Output***: the vocabulary.

```python
vocab = {
    "a": 0,
    "b": 1,
    "c": 2,
    ...
}
```

In [4]:
paths = [
    "dataset/crohme2019_train.txt",
    "dataset/crohme2019_test.txt",
    "dataset/crohme2019_valid.txt",
]

df = pd.read_csv(paths[1], sep="\t", header=None, names=["path", "label"]).dropna().astype(str)

df.head()

,path,label
0,crohme2019/test/UN19_1032_em_455.inkml,4 Right n Right - Right 4 Right ) NoRel ( NoRe...
1,crohme2019/test/UN19_1044_em_632.inkml,\phi Sub 0 NoRel = Right d Right x Sup 1 Right...
2,crohme2019/test/UN19wb_1111_em_1039.inkml,9 Right \times Right 9 Right + Right 1 Right 3...
3,crohme2019/test/UN19_1028_em_394.inkml,\tan Right \alpha Right \tan Right \theta Sup ...
4,crohme2019/test/UN19_1005_em_64.inkml,a Right = NoRel 1 NoRel - Below \sqrt Inside 2...


In [5]:
def get_unique_chars(paths) -> set:
    res = set()
    for i in df['label'].apply(lambda x: x.strip().split()):
        res.update(i)
    return res

len(get_unique_chars(paths[0]))

108

In [ ]:
unique_chars = get_unique_chars(paths[0])
unique_chars

The vocabulary should be sorted by ascii table. The first character (index 0) must be blank (`''`).

In [6]:
def chr2idx(unique_chars: set) -> dict:
    unique_chars.add('')
    return {char: idx for idx, char in enumerate(sorted(unique_chars))}

vocab = chr2idx(get_unique_chars(paths[0]))

assert vocab[""] == 0
assert vocab["|"] == 108
assert vocab["\\pi"] == 68
assert vocab["\\exists"] == 51
assert len(vocab) == 109

In [10]:
class Vocab:
    """

    Attributes:
        paths (list): A list of file paths to the annotation files.
        char2idx (dict): A dictionary mapping characters to their corresponding indices.  This is the primary vocabulary.
        idx2char (dict): A reverse mapping from indices to characters, facilitating decoding.

    Methods:
        build_vocab(): Builds the vocabulary from the specified annotation files, sorting characters lexicographically and adding a blank character.
        get_vocab(): Returns the character-to-index vocabulary.
        save_vocab(path): Saves the vocabulary to a JSON file.
        encode(tokens): Converts a list of tokens (characters) into a list of their corresponding indices.
        decode(ids): Converts a list of indices back into a list of tokens (characters).
    """
    def __init__(self, vocab_file=None):
        self.char2idx = {}
        if vocab_file is not None:
            self.char2idx = json.load(open(vocab_file))
        self.idx2char = {v: k for k, v in self.char2idx.items()}

    def build_vocab(self, annotations) -> None:
        # TODO: build vocab from all annotation files
        vocab = set()
        for annotation in annotations:
            vocab.update(get_unique_chars(annotation))
        vocab = chr2idx(vocab)

        # assign vocab to the class
        self.char2idx = vocab

    def get_vocab(self) -> dict:  # getter
        return self.char2idx

    def save_vocab(self, path: str) -> None:  # save vocab to json file
        with open(path, 'w') as f:
            json.dump(self.char2idx, f)

    def encode(self, tokens):
        return [self.char2idx[token] for token in tokens]

    def decode(self, ids):
        return [self.idx2char[id] for id in ids]


In [11]:
# CHECKPOINT: build vocab from annotations
annotations = [
    "dataset/crohme2019_train.txt",
    "dataset/crohme2019_test.txt",
    "dataset/crohme2019_valid.txt",
]

vocab = Vocab()
vocab.build_vocab(annotations)
vocab.save_vocab('vocab.json')

In [12]:
# TEST: vocab class loaded from prebuilt JSON
vocab = Vocab('vocab.json')

# encoding a sequence
input = '- Right \\sqrt Inside 2'.split()

assert vocab.encode(input) == [5, 37, 74, 30, 10]
assert vocab.decode(vocab.encode(input)) == input

## Task 2: Build Dataset

In this task, we will build a dataset class that will be used to load the dataset files _(train, valid, and test)_.

The dataset class will also be used to preprocess the data by converting the characters in the data to integers using the vocabulary.

> _Before going to this task, it is **highly recommended** to read this [Tutorial on creating Custom Dataset](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files) with PyTorch._
>
> _You should have basic understanding of required methods in the Dataset class such as `__init__`, `__len__`, and `__getitem__`._


***Input***: Dataset class should take the following arguments:.

- `annotation`: the path to the dataset annotation *(`.txt`)*.
- `root_dir`: the root directory of the dataset file.
- `vocab`: the vocabulary.

***Output***: the dataset class.

> *Note: You can adjust the Dataset class base on your references.*


### Dataflow
Below is a typical dataflow to train a model.
```mermaid
flowchart LR
    A[Raw Data] --> |parse| B[Cleaned Data]
    B --> |feature engineering| C[Dataset]
    C --> |package| D[DataLoader]
    D --> |load| E[Model]
    E --> |predict| F[Output]
    F --> |calculate| G[Loss]
    G --> |backpropagate| E
```

### Handling Inkml files

We have an inkml file: `crohme2019/test/UN19_1041_em_597.inkml`

Its label is `- Right \sqrt Inside 2`

Target would be: `[5, 37, 74, 30, 10]`, where the index here is the index of tokens (from a vocabulary)

We provided you with class `Inkml` to handle the parsing and processing of the inkml files. The main method you will use is `getTraces()` which returns the traces of the inkml file.

> _Inspect this class and understand how you should be working with this type of data._

```python

In [13]:
class Segment(object):
    """Class to reprsent a Segment compound of strokes (id) with an id and label."""

    __slots__ = ("id", "label", "strId")

    def __init__(self, *args):
        if len(args) == 3:
            self.id = args[0]
            self.label = args[1]
            self.strId = args[2]
        else:
            self.id = "none"
            self.label = ""
            self.strId = set([])


class Inkml(object):
    """Class to represent an INKML file with strokes, segmentation and labels"""

    __slots__ = ("fileName", "strokes", "strkOrder", "segments", "truth", "UI")

    NS = {
        "ns": "http://www.w3.org/2003/InkML",
        "xml": "http://www.w3.org/XML/1998/namespace",
    }

    def __init__(self, *args):
        self.fileName = None
        self.strokes = {}
        self.strkOrder = []
        self.segments = {}
        self.truth = ""
        self.UI = ""
        if len(args) == 1:
            self.fileName = args[0]
            self.loadFromFile()

    def fixNS(self, ns, att):
        """Build the right tag or element name with namespace"""
        return "{" + Inkml.NS[ns] + "}" + att

    def loadFromFile(self):
        """load the ink from an inkml file (strokes, segments, labels)"""
        tree = ET.parse(self.fileName)

        # # ET.register_namespace();
        root = tree.getroot()

        for info in root.findall("ns:annotation", namespaces=Inkml.NS):
            if "type" in info.attrib:
                if info.attrib["type"] == "truth":
                    self.truth = info.text.strip()
                if info.attrib["type"] == "UI":
                    self.UI = info.text.strip()

        for strk in root.findall("ns:trace", namespaces=Inkml.NS):
            self.strokes[strk.attrib["id"]] = strk.text.strip()
            self.strkOrder.append(strk.attrib["id"])
        segments = root.find("ns:traceGroup", namespaces=Inkml.NS)

        if segments is None or len(segments) == 0:
            return

        for seg in segments.iterfind("ns:traceGroup", namespaces=Inkml.NS):
            id = seg.attrib[self.fixNS("xml", "id")]
            label = seg.find("ns:annotation", namespaces=Inkml.NS).text
            strkList = set([])
            for t in seg.findall("ns:traceView", namespaces=Inkml.NS):
                strkList.add(t.attrib["traceDataRef"])
            self.segments[id] = Segment(id, label, strkList)

    def getTraces(self, height=256):
        traces_array = [
            np.array(
                [p.strip().split() for p in self.strokes[id].split(",")], dtype="float"
            )
            for id in self.strkOrder
        ]

        ratio = height / (
            (
                np.concatenate(traces_array, 0).max(0)
                - np.concatenate(traces_array, 0).min(0)
            )[1]
            + 1e-6
        )
        return [(trace * ratio).astype(int).tolist() for trace in traces_array]

    def view(self):
        plt.figure(figsize = (16, 4))
        plt.axis("off")
        for trace in self.getTraces():
            trace_arr = np.array(trace)
            plt.plot(trace_arr[:, 0], -trace_arr[:, 1])  # invert y coordinate

In [ ]:
inkml_path = 'dataset/crohme2019/crohme2019/test/ISICal19_1201_em_758.inkml'
ink = Inkml(inkml_path)
ink.view()

In [ ]:
# Display shapes of first 5 traces in one file

[np.array(trace).shape for trace in ink.getTraces()][:5]

> **REMARK**: _Some Inkml may return (x, y, t) coordinates, while most of them return (x, y) coordinates. Make sure to handle this inconsistency in your implementation._

### Build Dataset

For calculating the Connectionist Temporal Classification (CTC) loss, read the [expected output](https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html) of this loss function from PyTorch.

> _Hint: focus on the shapes and the expected variables. (see code example for calculating loss)_
> ```python
> loss = ctc_loss(input_tensor, target_tensor, input_lengths, target_lengths)
> ```

**Base on the observation, how can we build a dataset class from the dataset files?**

> Hints:
>   - *Understand the big picture. (Dataflow)*
>   - *What informations needed to calculate our Loss?*
>   - *What should be returned in the `__getitem__` method?*
>   - *The outputs should be of Tensor type*
   
***References from pytorch [tutorials](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html)***

`torch.utils.data.Dataset` is an abstract class representing a dataset. Your custom dataset should inherit `Dataset` and override the following methods:

- `__len__` so that `len(dataset)` returns the size of the dataset.

- `__getitem__` to support the indexing such that `dataset[i]` can be used to get ith sample.


### Feature representation

In this task, we will convert the strokes into feature representations that can be used by the model.

Initially, the `.getTraces()` method returns list of strokes, where each stroke is a list of (x, y) coordinates. Intuitively, we can use those coordinates as features. But if directly using the coordinates as features, the model will have a hard time learning the patterns.

Instead, we can calculate the difference *(∆d)* between consecutive coordinates as features. This way, the model can learn the patterns more easily.

$\Delta_x = x_{i+1} - x_i$ and $\Delta_y = y_{i+1} - y_i$

The feature would then be normalized as

($\frac{\Delta_x}{d}$ , $\frac{\Delta_y} {d}$), where $d = \sqrt{\Delta_x^2 + \Delta_y^2}$

**Pen-up and Pen-down**

In the dataset, each stroke is separated by a pen-up event. We can use this information to separate the strokes.

1. The pen is lifted from the paper (connecting the end of a stroke to the start of a stroke): $pen\_up = 1$
2. The pen is on the paper: $pen\_up = 0$

Then, our feature representation would be:
($\frac{\Delta_x}{d}$ , $\frac{\Delta_y} {d}$, $d$, $pen\_up$)


```
point 1: (x1, y1)
point 2: (x2, y2)
point 3: (x3, y3) <--- end of stroke #1
point 4: (x4, y4) <--- start of stroke #2
...
point n: (xn, yn)
```

The feature representation will be:

```
f1 = ((x2 - x1) / d, (y2 - y1)/d, d, 0)
f2 = ((x3 - x2) / d, (y3 - y2)/d, d, 0)
f3 = ((x4 - x3) / d, (y4 - y3)/d, d, 1) <-- pen up
...
fn-1 = ((xn - xn-1) / d, (yn - yn-1)/d, d, 0)
```

#### **CHECK LIST**

For each data sample, we will do the following steps
- Combine all the strokes into a single stroke (N, 2)
- Compute first order differences of x and y coordinates
- Remove any zero length strokes
- Compute Euclidean distances between consecutive points
- Normalize the x and y coordinates by Euclidean distance
- Add feature pen-up/pen-down
> *Features of transformed data: (delta traces, distance, pen_up_down)*

Label
- Define label (list of indices of the words)

Finally, we will convert the data and label to PyTorch tensors.
- Convert data and label to PyTorch tensors

In [14]:
class InkmlDataset(Dataset):
    def __init__(self, annotation, root_dir, vocab, augment=False, augment_prob=0.5):
        """
        Arguments:
            annotation (string): annotation file txt.
            root_dir (string): directory holds the dataset.
            vocab (set): of vocab.
        """
        self.annotation = annotation
        self.root_dir = root_dir
        self.vocab = vocab
        self.augment = augment
        self.augment_prob = augment_prob

        # load annotations
        self.inks = []
        self.labels = []

        # your code to create self.inks and self.labels from the annotation file
        with open(f"{self.root_dir}/{self.annotation}", 'r') as f:
            for line in f:
                # Take apart each line and clean them
                parts = line.strip().split('\t')

                # Set a path to the inkml file
                inkml_path = os.path.join(self.root_dir, "crohme2019", parts[0])

                # Populate arrays
                label = parts[1].strip().split()
                self.inks.append(inkml_path)
                self.labels.append(label)

    def apply_rotation(self, traces, angle_range=(-15, 15)):

        if not self.augment or np.random.random() > 0.4:  # 40% chance
            return traces

        angle = np.random.uniform(angle_range[0], angle_range[1]) * np.pi / 180
        cos_a, sin_a = np.cos(angle), np.sin(angle)
        rotation_matrix = np.array([[cos_a, -sin_a], [sin_a, cos_a]])

        augmented_traces = []

        # Calculate global centroid of all traces for consistent rotation
        all_points = []
        for trace in traces:
            if len(trace) > 0:
                all_points.extend(trace)

        if len(all_points) == 0:
            return traces

        global_centroid = np.mean(all_points, axis=0)

        for trace in traces:
            if len(trace) == 0:
                augmented_traces.append(trace)
                continue

            trace_array = np.array(trace, dtype=np.float32)
            # Rotate around global centroid to maintain relative positions
            rotated = (trace_array - global_centroid) @ rotation_matrix.T + global_centroid
            augmented_traces.append(rotated.tolist())

        return augmented_traces

    def apply_scaling(self, traces, scale_range=(0.85, 1.15)):

        if not self.augment or np.random.random() > 0.5:  # 50% chance
            return traces

        # Use same scale for both x and y to maintain aspect ratio
        scale = np.random.uniform(scale_range[0], scale_range[1])

        augmented_traces = []

        # Calculate global centroid of all traces for consistent scaling
        all_points = []
        for trace in traces:
            if len(trace) > 0:
                all_points.extend(trace)

        if len(all_points) == 0:
            return traces

        global_centroid = np.mean(all_points, axis=0)

        for trace in traces:
            if len(trace) == 0:
                augmented_traces.append(trace)
                continue

            trace_array = np.array(trace, dtype=np.float32)
            # Scale around global centroid
            scaled = (trace_array - global_centroid) * scale + global_centroid
            augmented_traces.append(scaled.tolist())

        return augmented_traces

    def apply_augmentation(self, traces):
        """Apply augmentation pipeline"""
        if not self.augment or np.random.random() > self.augment_prob:
            return traces

        # Apply transformations in sequence
        augmented_traces = traces

        # Apply rotation first
        augmented_traces = self.apply_rotation(augmented_traces)

        # Then apply scaling
        augmented_traces = self.apply_scaling(augmented_traces)

        return augmented_traces

    def __len__(self):
        """This code should return the number of samples in the dataset"""
        return len(self.labels)

    def __getitem__(self, idx):
        """This code should return the idx-th sample in the dataset"""
        ink = self.inks[idx]
        label = self.labels[idx]

        ink_content = Inkml(ink)
        traces = ink_content.getTraces()

        # Apply augmentation to traces
        traces = self.apply_augmentation(traces)

        combined_stroke = []
        pen_up_markers = []

        for stroke_id, stroke in enumerate(traces):
            if not stroke:
                continue

            for point in stroke:
                # Ensure we always get exactly 2 coordinates (x, y)
                if len(point) >= 2:
                    # Take only x, y coordinates
                    combined_stroke.append([float(point[0]), float(point[1])])
                    pen_up_markers.append(0)
                else:
                    continue

            # Mark pen-up for last point of each stroke (except final stroke)
            if stroke_id < len(traces) - 1 and len(combined_stroke) > 0:
                pen_up_markers[-1] = 1

        # Convert to arrays
        pt_array = np.array(combined_stroke, dtype=np.float32)
        pen_up_pts = np.array(pen_up_markers, dtype=np.float32)

        # Calculate differences
        delta = np.diff(pt_array, axis=0)
        pen_up_diffs = pen_up_pts[1:]

        # Calculate distances and filter
        e_dist = np.sqrt(np.sum(delta**2, axis=1))
        non_zero_mask = (e_dist > 1e-8)

        # Remove 0-length strokes
        delta = delta[non_zero_mask]
        e_dist = e_dist[non_zero_mask]
        pen_up_feature = pen_up_diffs[non_zero_mask]

        # Normalize and avoid division by zero
        e_dist_safe = np.maximum(e_dist, 1e-8)
        normal_delta = delta / e_dist_safe[:, np.newaxis]
        pen_up_feature = pen_up_feature[:, np.newaxis]

        input_features = np.concatenate((
            normal_delta,
            e_dist[:, np.newaxis],
            pen_up_feature
        ), axis=1)

        # Convert to tensors
        input_tensor = torch.from_numpy(input_features).float()
        input_len = torch.tensor(input_features.shape[0], dtype=torch.long)

        # Encode label tokens to indices
        target_indices = self.vocab.encode(label)
        target_tensor = torch.tensor(target_indices, dtype=torch.long)
        label_len = torch.tensor(len(target_indices), dtype=torch.long)

        return input_tensor, target_tensor, input_len, label_len

## Task 3: Build Lightning Data Module via Dataloader

Revise [this tutorial](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#preparing-your-data-for-training-with-dataloaders) to see how you can prepare your data for training with DataLoader.

Then refer to this [documentation](https://pytorch.org/docs/stable/data.html#loading-batched-and-non-batched-data) to understand how DataLoader in PyTorch loads Batched or Non-Batched data.

> **TASK:** Write the dataloader with custom collate function to pad the input and target sequence

For a better understanding of the importance of handling variable-length sequences in deep learning models, it is crucial to format and pad the data appropriately to ensure consistency during training. Learn more about this topic [here](https://plainenglish.io/blog/understanding-collate-fn-in-pytorch-f9d1742647d3).

> _**Hint 1**: use `torch.nn.utils.rnn.pad_sequence` to pad the input and target sequences. Read more about this function [here](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html)._

> _**Hint 2**: read Example 3 from this [tutorial](https://www.programiz.com/python-programming/methods/built-in/zip) to unpack data using `zip()`._

In [15]:
def collate_fn(batch): # Expects a list of tuples of tensors
    """Create batch"""
    # TODO: your custom colate function here
    features, labels, input_lens, label_lens = zip(*batch)
    features = nn.utils.rnn.pad_sequence(features, batch_first=True)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True) # (N, S)
    # features, labels, input_lens, label_lens should be torch.tensor
    input_lens = torch.stack(input_lens)
    label_lens = torch.stack(label_lens)

    return features, labels, input_lens, label_lens

In [ ]:
dataset = InkmlDataset("crohme2019_train.txt", "dataset", vocab)  # TODO: initialize a dataset
features, labels, input_lens, label_lens = collate_fn([dataset[0], dataset[1]])

import numpy.testing as npt

assert type(input_lens) == torch.Tensor
assert type(label_lens) == torch.Tensor

> **TASK:** implement InkmlDataset_PL Lightning Datamodule

In [16]:
class InkmlDataset_PL(pl.LightningDataModule):
    """
    PyTorch Lightning data module for handling the INKML dataset.
    """

    def __init__(
        self,
        batch_size: int = 10,
        workers: int = 5,
        train_data: str = "",
        val_data: str = "",
        test_data: str = "",
        root_dir: str = "dataset",
        augment_train: bool = False,
        augment_prob: float = 0.6
    ):
        super().__init__()
        self.batch_size = batch_size
        self.workers = workers
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data
        self.root_dir = root_dir
        self.augment_train = augment_train
        self.augment_prob = augment_prob

    def setup(self, stage=None):
        if stage == "fit" or stage is None:
            self.train_dataset = InkmlDataset(
                self.train_data,
                root_dir=self.root_dir,
                vocab=Vocab("vocab.json"),
                augment=self.augment_train,
                augment_prob=self.augment_prob
            )
            self.val_dataset = InkmlDataset(
                self.val_data,
                root_dir=self.root_dir,
                vocab=Vocab("vocab.json"),
                augment=False  # Never augment validation data
            )
        if stage == "test" or stage is None:
            self.test_dataset = InkmlDataset(
                self.test_data,
                root_dir=self.root_dir,
                vocab=Vocab("vocab.json"),
                augment=False  # Never augment test data
            )

    def custom_collate_fn(self, data):
        # TODO: provide your custom collate function here.
        padded_traces, padded_labels, len_traces, len_labels = collate_fn(data)
        return padded_traces, padded_labels, len_traces, len_labels

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            num_workers=self.workers,
            collate_fn=self.custom_collate_fn,
            shuffle=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=self.workers,
            collate_fn=self.custom_collate_fn,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            num_workers=self.workers,
            collate_fn=self.custom_collate_fn,
        )

## Task 4: Build Model

In this task, we will build a model that will be used to train the dataset.
The model will be a simple RNN with a single layer of Bidirectional LSTM cells. The model will take the input from the dataset and output the predicted sequence of symbols and relations and will use CTC loss to train the model.

The model will be built using PyTorch and will use the following architecture


> _We highly recommend you to read through this [tutorial on Creating a model using Pytorch](https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html) and understand the basic building blocks of a model in PyTorch._

> _**ATTENTION** Below is the architecture for the model. You must **STRICTLY** follow the architecture and the parameters mentioned below._
> | Layer Type         | Configuration                              |
> |--------------------|--------------------------------------------|
> | Input              | Sequence of vectors with `input_size=4`    |
> | LSTM Layer         | `hidden_size=256`, `num_layers=2`          |
> |                    | `batch_first=True`                         |
> |                    | `bidirectional=True`                       |
> | LSTM Output        | Output shape: `(batch_size, seq_len, hidden_size*2)`|
> | Fully Connected    | `Linear(hidden_size*2, num_classes)`|
> | Activation         | `LogSoftmax(dim=...)`|
> | Output             | `(batch_size, seq_len, num_classes)`|


In [17]:
import torch
import torch.nn as nn

class LSTM_TemporalClassification(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM_TemporalClassification, self).__init__()
        # Define the Bidirectional LSTM layer
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True # Use a bidirectional LSTM
        )

        # Define the Linear layer (Fully Connected layer)
        # Bidirectional LSTM outputs 2 * hidden_size features
        self.fc = nn.Linear(hidden_size * 2, num_classes)

        # Define the LogSoftmax activation layer
        # Apply LogSoftmax over the class dimension
        self.log_softmax = nn.LogSoftmax(dim=2)

    def forward(self, x):
        # Pass the input through the LSTM layer
        # lstm_out shape: (batch_size, seq_len, hidden_size * 2) for bidirectional
        lstm_out, _ = self.lstm(x)

        # fc_out shape: (batch_size, seq_len, num_classes)
        fc_out = self.fc(lstm_out)

        # log_softmax_out shape: (batch_size, seq_len, num_classes)
        log_softmax_out = self.log_softmax(fc_out)

        return log_softmax_out

In [ ]:
# Test your implementation
model = LSTM_TemporalClassification(4, 256, 2, 109)
assert model.forward(torch.rand((10, 100, 4))).shape == (10, 100, 109)

## Task 5: Understand CTC Loss

In this task, we will understand how to use CTC loss to train the model. The CTC loss is used to train the model to predict the sequence of symbols and relations from the input sequence of features.

> _For deeper understanding of CTC loss, read this [blog](https://distill.pub/2017/ctc/)_

Check out this [Pytorch documentation](https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html) to learn more about its implementation.

> _Hint: read the expected input and output of the CTC loss function. The input should be of shape `(T, N, C)` where T is the length of the input sequence, N is the batch size, and C is the number of classes. The target should be of shape `(N, S)` where S is the length of the target sequence._

After understanding how CTC Loss works, we can proceed to implement it in our model.

## Task 6: Build Lightning Module

In this task, we will build a Lightning module that will be used to train the model. The Lightning module will mostly be used to define the training and validation steps, as well as the optimizer and learning rate scheduler.

> _More on building Lightning module can be found [here](https://lightning.ai/docs/pytorch/stable/common/lightning_module.html). You should read about core methods to know what to implement in your module._

In [24]:
class MathOnlineModel(pl.LightningModule):
    def __init__(
        self,
        lr=0.001,
        input_size=4,
        output_size=109,
        hidden_size=256,
        num_layers=2,
        decoder=None,
        constraint_weight=0.1,
    ):
        super().__init__()
        self.model = LSTM_TemporalClassification(
            input_size, hidden_size, num_layers, output_size
        )
        self.criterion = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)
        self.lr = lr
        self.constraint_weight = constraint_weight

        # Load vocab and decoder
        self.vocab = Vocab("vocab.json")
        self.greedy_decoder = GreedyCTCDecoder(self.vocab) if decoder is None else decoder

        # Relation tokens for constraint loss
        self.RELATIONS = ['Right', 'Above', 'Below', 'Sup', 'Sub', 'Inside', 'NoRel']
        self.relation_indices = self._get_relation_indices()

        # For validation WER calculation
        self.val_wer_samples = []
        self.max_wer_samples = 50

    def _get_relation_indices(self):
        """Get indices of relation tokens in vocabulary"""
        relation_indices = [self.vocab.char2idx[rel] for rel in self.RELATIONS if rel in self.vocab.char2idx]
        return torch.tensor(relation_indices, dtype=torch.long)

    def forward(self, x):
        return self.model(x)

    def _calculate_constraint_loss(self, log_probs, input_features, input_lengths):
        """Calculate constraint loss to penalize relations at pen-down positions"""
        batch_size, seq_len, num_classes = log_probs.shape
        device = log_probs.device
        relation_indices = self.relation_indices.to(device)
        total_constraint_loss = 0.0

        for i in range(batch_size):
            try:
                actual_len = input_lengths[i].item()
                if actual_len <= 0 or actual_len > seq_len:
                    continue

                # Extract pen-up/pen-down information (4th feature)
                pen_up_states = input_features[i, :actual_len, 3]
                pen_down_mask = (pen_up_states == 0).float()

                # Skip if contains NaNs
                if torch.isnan(pen_down_mask).any():
                    continue

                log_probs_sample = log_probs[i, :actual_len, :]
                if torch.isnan(log_probs_sample).any():
                    continue

                # Convert to probabilities safely
                log_probs_clamped = torch.clamp(log_probs_sample, min=-50, max=50)
                probs = torch.exp(log_probs_clamped)

                # Get relation probabilities
                if len(relation_indices) > 0:
                    relation_probs = probs[:, relation_indices]
                    p_rel = torch.sum(relation_probs, dim=1)
                else:
                    p_rel = torch.zeros(actual_len, device=device)

                # Calculate constraint loss: -log(1 - sum(p_rel * pen_down))
                constraint_term = torch.sum(p_rel * pen_down_mask)
                constraint_term = torch.clamp(constraint_term, min=0.0, max=actual_len)

                eps = 1e-8
                log_arg = torch.clamp(1.0 - constraint_term + eps, min=eps, max=1.0)
                constraint_loss = -torch.log(log_arg)

                if not (torch.isnan(constraint_loss) or torch.isinf(constraint_loss)):
                    total_constraint_loss += constraint_loss

            except Exception:
                continue

        return total_constraint_loss / max(batch_size, 1)

    def _calculate_wer_metrics(self, log_probs, targets, input_lens, target_lens):
        """Calculate WER metrics for a single sample"""
        try:
            seq_len = input_lens[0].item()
            target_len = target_lens[0].item()

            # Decode prediction
            emission = log_probs[0, :seq_len, :]
            with torch.no_grad():
                pred_tokens = self.greedy_decoder.forward(emission)

            # Get ground truth tokens
            target_indices = targets[0, :target_len].cpu().numpy()
            target_tokens = self.vocab.decode(target_indices.tolist())

            # Calculate edit distances
            total_ed = edit_distance(pred_tokens, target_tokens)
            total_len = len(target_tokens)

            # Separate symbols and relations
            pred_syms = [p for p in pred_tokens if p not in self.RELATIONS]
            pred_rels = [p for p in pred_tokens if p in self.RELATIONS]
            target_syms = [t for t in target_tokens if t not in self.RELATIONS]
            target_rels = [t for t in target_tokens if t in self.RELATIONS]

            sym_ed = edit_distance(pred_syms, target_syms)
            sym_len = len(target_syms)
            rel_ed = edit_distance(pred_rels, target_rels)
            rel_len = len(target_rels)

            return {
                'total_ed': total_ed, 'total_len': total_len,
                'sym_ed': sym_ed, 'sym_len': sym_len,
                'rel_ed': rel_ed, 'rel_len': rel_len
            }
        except Exception:
            return {'total_ed': 0, 'total_len': 1, 'sym_ed': 0, 'sym_len': 1, 'rel_ed': 0, 'rel_len': 1}

    def training_step(self, batch, batch_idx):
        x, y, x_lens, y_lens = batch

        # Check for NaN inputs
        if torch.isnan(x).any():
            return None

        # Forward pass
        log_probs = self.forward(x)
        log_probs_ctc = log_probs.transpose(0, 1)  # (T, N, C) for CTC loss

        if torch.isnan(log_probs_ctc).any():
            return None

        # Calculate losses
        ctc_loss = self.criterion(log_probs_ctc, y, x_lens, y_lens)
        constraint_loss = self._calculate_constraint_loss(log_probs, x, x_lens)
        total_loss = ctc_loss + self.constraint_weight * constraint_loss

        # Handle bad loss values
        if torch.isnan(total_loss) or torch.isinf(total_loss):
            return None

        return total_loss

    def validation_step(self, batch, batch_idx):
        x, y, x_lens, y_lens = batch

        # Forward pass
        log_probs = self.forward(x)
        log_probs_ctc = log_probs.transpose(0, 1)

        # Calculate losses
        ctc_loss = self.criterion(log_probs_ctc, y, x_lens, y_lens)
        constraint_loss = self._calculate_constraint_loss(log_probs, x, x_lens)
        total_loss = ctc_loss + self.constraint_weight * constraint_loss

        # Log only val_loss (no val_ctc_loss)
        self.log("val_loss", total_loss, prog_bar=True, on_epoch=True)

        # Calculate WER for limited samples
        if len(self.val_wer_samples) < self.max_wer_samples:
            try:
                wer_data = self._calculate_wer_metrics(
                    log_probs[0:1], y[0:1], x_lens[0:1], y_lens[0:1]
                )
                self.val_wer_samples.append(wer_data)
            except Exception:
                pass

        return total_loss

    def on_validation_epoch_end(self):
        """Calculate and log WER metrics after validation epoch"""
        if len(self.val_wer_samples) > 0:
            # Calculate metrics
            total_ed = sum(s['total_ed'] for s in self.val_wer_samples)
            total_len = sum(s['total_len'] for s in self.val_wer_samples)
            val_wer = total_ed / max(total_len, 1)

            sym_ed = sum(s['sym_ed'] for s in self.val_wer_samples)
            sym_len = sum(s['sym_len'] for s in self.val_wer_samples)
            val_wer_sym = sym_ed / max(sym_len, 1)

            rel_ed = sum(s['rel_ed'] for s in self.val_wer_samples)
            rel_len = sum(s['rel_len'] for s in self.val_wer_samples)
            val_wer_rel = rel_ed / max(rel_len, 1)

            # Log WER metrics
            self.log("val_wer", val_wer, prog_bar=True)
            self.log("val_wer_sym", val_wer_sym, prog_bar=True)
            self.log("val_wer_rel", val_wer_rel, prog_bar=True)

        # Reset for next epoch
        self.val_wer_samples = []

    def test_step(self, batch, batch_idx):
        x, y, x_lens, y_lens = batch

        # Forward pass
        log_probs = self.forward(x)
        log_probs_ctc = log_probs.transpose(0, 1)

        # Calculate losses
        ctc_loss = self.criterion(log_probs_ctc, y, x_lens, y_lens)
        constraint_loss = self._calculate_constraint_loss(log_probs, x, x_lens)
        total_loss = ctc_loss + self.constraint_weight * constraint_loss

        return total_loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.model.parameters(),
            lr=self.lr,
            weight_decay=1e-4
        )

        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.5,
            patience=2,
            min_lr=1e-6
        )

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
                "interval": "epoch"
            }
        }

## Task 7: Train the Model with PyTorch Lightning

Read more about Training with PyTorch Lightning [here](https://lightning.ai/docs/pytorch/stable/common/trainer.html) and understand how to use the Trainer class to train the model.

From the documentation:

> The Lightning Trainer does much more than just “training”. Under the hood, it handles all loop details for you, some examples include:
> - Automatically enabling/disabling grads
> - Running the training, validation and test dataloaders
> - Calling the Callbacks at the appropriate times
> - Putting batches and computations on the correct devices

> **IMPORTANT**: You must config your WandB logger to log the training and validation metrics. Without this, your work will not be graded.

In [18]:
student_id = "10423091"  # TODO: replace with your student ID

# wandb api key is stored in Colab Secrets
api_key = userdata.get('WANDB_API_KEY')
os.environ['VAR_NAME'] = api_key
# api_key = os.environ.get("WANDB_API_KEY", "")  # configure your wandb key

# if api_key == "":
#     raise ValueError("Please set your wandb key in the code or in the environment variable WANDB_API_KEY")
# else:
#     print("WandB API key is set. Proceeding with login...")

wandb.login(key=os.environ['VAR_NAME'])

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: phucdoan_10423091 (phucdoan_10423091-vietnamese-german-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
ckpt_path = '/content/drive/MyDrive/crohme2019/epoch=18-val_loss=0.6517.ckpt'

# Define shared parameters once
TRAINING_PARAMS = {
    "batch_size": 32,
    "max_epochs": 20,
    "hidden_size": 256,
    "num_layers": 2,
    "input_size": 4,
    "output_size": 109,
    "workers": 8,
    "learning_rate": 0.001,
    "monitoring_metric": "val_wer",
}

wandb_logger = WandbLogger(
    entity="cuong-nt-vgu-ai-2025", ## DO NOT CHANGE
    project="math_online_2025", ## DO NOT CHANGE
    name=f"{student_id}_run_1",
    config={
        "student_id": student_id, ## DO NOT CHANGE
        "model": "LSTM_TemporalClassification",
        **TRAINING_PARAMS  # Unpack all training parameters
    },
    log_model=True,
    save_dir="wandb_logs",
)

trainer = Trainer(
    callbacks=[
        LearningRateMonitor(logging_interval='step'),
        ModelCheckpoint(
            filename='{epoch}-{val_wer:.4f}',
            save_top_k=5,
            monitor=TRAINING_PARAMS["monitoring_metric"],
            mode='min'
        ),
    ],
    logger=wandb_logger,
    check_val_every_n_epoch=1,
    fast_dev_run=False, # enable for testing
    default_root_dir='checkpoint',
    precision="16-mixed",
    deterministic=False,
    max_epochs=TRAINING_PARAMS["max_epochs"],
    log_every_n_steps=50,
    accelerator="gpu",
    devices="auto",
    gradient_clip_val=1,
    accumulate_grad_batches=2
)

model = MathOnlineModel()
# model = MathOnlineModel_Constraint()
# model = MathOnlineModel.load_from_checkpoint(ckpt_path)

dm = InkmlDataset_PL(
    root_dir='dataset',
    train_data='crohme2019_train.txt',
    val_data='crohme2019_valid.txt',
    test_data='crohme2019_test.txt',
    batch_size=TRAINING_PARAMS["batch_size"],
    workers=TRAINING_PARAMS["workers"],
    augment_train=True,
    augment_prob=0.
)

trainer.fit(model, dm)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type                        | Params | Mode 
-----------------------------------------------------------------------
0 | model          | LSTM_TemporalClassification | 2.2 M  | train
1 | criterion      | CTCLoss                     | 0      | trai

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

**Requirements:**
- Validation loss (val_loss < 0.7)

**Tips on training:**
- Debug with **fast_dev_run**:
Before run the training process, it is better to make a quick check of training and validation loop. Set fast_dev_run=True, then run the trainer and check if there is any bug exist.
- Training initially with **small batch size**:
In practice, training with CTC loss converges slowly for large batch size. To accelerate convergence, training with small batch size first, save model, then, train with large batch size.
- Train more epochs by setting:

```python
trainer = Trainer(
    ...
    max_epochs=20,
    ...
)
```

## Task 8: Test your model

Run the test set and check the accuracy of your model. The test set is used to evaluate the performance of the model on unseen data.

In [ ]:
trainer = Trainer(
    devices=1,
)

# Load the model from a checkpoint
model = MathOnlineModel.load_from_checkpoint(
    ckpt_path,
)

# Initialize the data module
dm = InkmlDataset_PL(
    root_dir="dataset",
    train_annotation="crohme2019_train.txt",
    validation_annotation="crohme2019_valid.txt",
    test_annotation="crohme2019_test.txt",
    vocab_file="vocab.json",
    batch_size=32,
    num_workers=4
)

# Test the model
trainer.test(model, datamodule=dm)

## Task 9: Inference

The output of networks need to process by a decoding step.

- Greedy decode: Your task is to implement greedy decoding method. Which converts the output into a string of symbols and relations (same form with labels). Greedy decoder produce the best path by removing consecutive repeated symbols/relations and then remove \<blank\>

> **TASK**: Implement the greedy decoder for the model output.

Based on GreedyCTCDecoder class from this [link](https://pytorch.org/audio/main/tutorials/asr_inference_with_ctc_decoder_tutorial.html#greedy-decoder), write decoding for an output. Your implementation should handle the output of the model and convert it into a string of symbols and relations, ensuring to remove consecutive repeated symbols/relations and the blank token.

In [19]:
class GreedyCTCDecoder(torch.nn.Module):
    def __init__(self, vocab):
        super().__init__()
        self.vocab = vocab
        self.blank = vocab.char2idx[""]

    def forward(self, emission: torch.Tensor) -> list:
        """Given a sequence emission over labels, get the best path
        Args:
          emission (Tensor): Logit tensors. Log probs from model. Shape `[seq_len, num_label]`.

        Returns:
          list: The resulting transcript
        """
        # TODO: implement the decoder
        # Get most likely token at rach timestep
        indices = torch.argmax(emission, dim=-1)
        indices = indices.tolist()
        # Decodes
        output_seq_list = []
        prev_idx = None

        for idx in indices:
            if idx != prev_idx and idx != self.blank:
                # Convert index back to readable token
                token = self.vocab.idx2char[idx]
                output_seq_list.append(token)
            prev_idx = idx

        return output_seq_list

In [20]:
class BeamSearchCTCDecoder(torch.nn.Module):
    def __init__(self, vocab, beam_size=5):
        super().__init__()
        self.vocab = vocab
        self.blank = vocab.char2idx[""]
        self.beam_size = beam_size

    def forward(self, emission: torch.Tensor) -> list:

        T, V = emission.shape

        # Initialize beam: (log_prob, prefix, last_char)
        beam = [(0.0, [], self.blank)]

        for t in range(T):
            candidates = []

            for log_prob, prefix, last_char in beam:
                for c in range(V):
                    new_log_prob = log_prob + emission[t, c].item()

                    if c == self.blank:
                        candidates.append((new_log_prob, prefix, c))
                    elif c == last_char:
                        candidates.append((new_log_prob, prefix, c))
                    else:
                        new_prefix = prefix + [self.vocab.idx2char[c]]
                        candidates.append((new_log_prob, new_prefix, c))

            # Keep top beam_size candidates
            candidates.sort(key=lambda x: x[0], reverse=True)
            beam = candidates[:self.beam_size]

        # Return best sequence
        return beam[0][1]

Then you can test the output of your model here

In [ ]:
model = MathOnlineModel.load_from_checkpoint(ckpt_path)
model.eval()

dataset = InkmlDataset("crohme2019_train.txt", "dataset", Vocab("vocab.json"))
feature, label, input_len, label_len = dataset.__getitem__(0)

vocab = Vocab("Vocab.json")

In [ ]:
decoder = GreedyCTCDecoder(vocab)

In [ ]:
decoder = BeamSearchCTCDecoder(vocab, beam_size=5)

In [ ]:
# Run inference
with torch.no_grad():
    # Add batch dimension to feature
    feature_batch = feature.unsqueeze(0)  # Shape: [1, seq_len, 4]
    # Get model predictions
    log_probs = model(feature_batch)  # [1, seq_len, 109]

    # Remove batch dimension for decoder
    emission = log_probs.squeeze(0)  # [seq_len, 109]
    # Decode the output
    decoded_output = decoder.forward(emission)

decoded_output = decoder.forward(emission)

print("Output:", decoded_output)
# possible output if your training work well
# decoded -> ['\\phi', 'Right', '(', 'Right', '0', 'Right', '(', 'Right', 'n', 'Right', ')', 'Right', ')']

## Task 10: Implement calculation metric for training

In [21]:
def edit_distance(pred_seq: list, label_seq: list):
    # TODO: implement Token Edit distance
    m, n = len(pred_seq), len(label_seq)

    # Create DP table
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # Initialize base cases
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    # Fill DP table
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if pred_seq[i-1] == label_seq[j-1]:
                dp[i][j] = dp[i-1][j-1]  # No operation needed
            else:
                dp[i][j] = 1 + min(
                    dp[i-1][j],    # Delete
                    dp[i][j-1],    # Insert
                    dp[i-1][j-1]   # Substitute
                )

    return dp[m][n]

assert (
    edit_distance(
        [
            "\\phi",
            "Right",
            "(",
            "Right",
            "0",
            "Right",
            "(",
            "Right",
            "n",
            "Right",
            ")",
            "Right",
            ")",
        ],
        [
            "\\phi",
            "Right",
            "(",
            "Right",
            "\\phi",
            "Right",
            "(",
            "Right",
            "n",
            "Right",
            ")",
            "Right",
            ")",
        ],
    )
    == 1
)

> **TASK**: Implement word error rate metric (wer) for training,  validation and testing in your model.
>
> $wer = \frac{total\ edit\ distance (predict\ sequence,\ target\ sequence)}{total\ target\ sequence\ length}$


**Steps to process:**

- Decode the predicted sequences to obtain the text output.
- Calculate the total edit distance between the predicted and target sequences.
- Compute the word error rate using the formula provided.
- Log the WER metric during training and validation.

**Continue to train the model**

- Load the latest trained model:

```
model = MathOnlineModel.load_from_checkpoint('path/to/your/checkpoint.ckpt')
```
- Change config of ModelCheckPoint to monitor the new metric (val_wer) instead.

```
        ModelCheckpoint(filename='{epoch}-{val_loss:.4f}', save_top_k=5, monitor='val_loss', mode='min')
        
      --> ModelCheckpoint(filename='{epoch}-{val_wer:.4f}', save_top_k=5, monitor='val_wer', mode='min'),

```


## **BONUS TASKS**

The following tasks are optional and can be done for extra credit to the final exam.

### **Bonus 1**: Add metric to evaluate accuracy of symbols and relations seperately *(+0.5pt)*

**Add the metrics: wer for symbols and wer for relations**
```
self.log('wer_sym',...)
self.log('wer_rel',...)
```

### **Bonus 2**: Modify loss function to constraint the output of relations at the time step of pen-up *(+1pt)*
Modify loss function to constrain the output of relations at the timestep of pen-up.

The idea: Provide a masked sequence such that the position of pen-up is masked, the additional loss would penaltize all the relations output to the timestep that has been masked.


**Loss function to constraint relation output**

```
pen_down: a masked sequence, where len(pen_down) = len(input_sequence)
        and pen_down[t] == 1 (pen down), pen_down[t] == 0 (pen up)
p_rel: total probability of relation outputs for every time step t, len(p_rel) = len(input_sequence), p_rel[t] = sum(p[t][rel] for rel in ['Sub', 'Sup', 'Above', ...])

The loss is defined as:

    L_{constraint} = -log(1 - sum(p_rel * pen_down))
    
Explanation:

    Minimize L_{constraint} (value range of (-inf, 0)) would make maximize of (1 - sum(p_rel * pen_down)), or making sum(p_rel * mask) -> 0, then it will penalize the relation output on pen_down timesteps.
```

**Apply constraint loss function with ctc loss**

```
    L = loss_{ctc} + \lambda * L_{constraint}
    
where \lambda is a weighted parameter to control balance between the two losses.
   
```


You can also find it in a published paper here, section 3.4:
https://arxiv.org/pdf/2105.10156

### **Bonus 3**: Visualize model prediction by timesteps and probability of softmax outputs *(+0.5pt)*

![alt text](https://github.com/fuisl/crohme-ctc/blob/6722b97ec2000afcd16068220f0b1b83b3134ff8/assets/graph_with_prob.png?raw=true)

### **Bonus 4**: Use CUDA CTC Decoder to optimize decoding process in model *(+0.5pt)*

Follow the instructions via this [link](https://pytorch.org/audio/main/tutorials/asr_inference_with_ctc_decoder_tutorial.html#cuda-ctc-decoder) to implement CUDA CTC decoder in your model.

In [ ]:
# Bonus point 3
def visualize_model_predictions_simple(model, feature, vocab):
    model.eval()

    with torch.no_grad():
        # batch dimension
        feature_batch = feature.unsqueeze(0)
        # log probabilities
        log_probs = model(feature_batch)

        # Convert to probabilities
        probs = torch.exp(log_probs.squeeze(0))  # [seq_len, num_classes]
        probs_np = probs.cpu().numpy()

    # Get predicted sequence
    predicted_indices = torch.argmax(log_probs.squeeze(0), dim=-1)
    predicted_tokens = [vocab.idx2char[idx.item()] for idx in predicted_indices]

    plt.figure(figsize=(16, 4))

    seq_len, num_classes = probs_np.shape

    # Plot maximum probability at each timestep
    max_probs = np.max(probs_np, axis=1)

    # Create line plot similar to reference image
    plt.plot(range(seq_len), max_probs, 'b-', linewidth=1)

    threshold = 0.4
    for t, (token, prob) in enumerate(zip(predicted_tokens, max_probs)):
        if token != '' and prob > threshold:
            plt.axvline(x=t, color='lightblue', alpha=0.7, linewidth=1)
            plt.text(t, -0.05, token, rotation=90, fontsize=8,
                    ha='center', va='top', color='red')

    plt.xlabel('Time Steps')
    plt.ylabel('Probability')
    plt.title('Model Predictions by Timesteps')
    plt.ylim(0, 1.0)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    return predicted_tokens

# Usage
def run_vis():
    model = MathOnlineModel.load_from_checkpoint(ckpt_path)
    vocab = Vocab("vocab.json")
    dataset = InkmlDataset("crohme2019_train.txt", "dataset", vocab)
    feature, label, input_len, label_len = dataset[0]

    predicted_tokens = visualize_model_predictions_simple(model, feature, vocab)
    print("Predicted sequence:", predicted_tokens)
    return predicted_tokens

predicted_tokens = run_vis()